In [14]:
from functions import *

dataset = load_dataset("/Users/connormcdonald/Desktop/Masters/MIT807/Data/twitter_training.csv", ['target', 't_id', 'created_at', 'query', 'user', 'text'])
print('dataset loaded')

dataset loaded


In [15]:
dataset = remove_unwanted_cols(dataset, ['t_id', 'created_at', 'query', 'user'])
dataset.text = dataset['text'].apply(preprocess_tweet_text)
print('dataset processed')

dataset processed


In [16]:
dataset.head()

,target,text
0,0,upset cant update Facebook texting might cry r...
1,0,I dived many times ball Managed save 50 The re...
2,0,whole body feels itchy like fire
3,0,behaving im mad I cant see
4,0,whole crew


In [3]:
# Same tf vector will be used for Testing sentiments on unseen trending data
tf_vector = get_feature_vector(np.array(dataset.iloc[:, 1]).ravel())
X = tf_vector.transform(np.array(dataset.iloc[:, 1]).ravel())
y = np.array(dataset.iloc[:, 0]).ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)
print('train test split complete')

train test split complete


In [4]:
# Training Naive Bayes model
NB_model = MultinomialNB()
LR_model = LogisticRegression(solver='lbfgs')
print('Naive Bayes model training')
NB_model.fit(X_train, y_train)
print('Logistic Regression model training')
LR_model.fit(X_train, y_train)

Naive Bayes model training
Logistic Regression model training


/Users/connormcdonald/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [5]:
print('Naive Bayesmodel validating')
y_predict_nb = NB_model.predict(X_test)
print(f'{accuracy_score(y_test, y_predict_nb)}\n')

print('Logistic Regression Model validating')
y_predict_lr = LR_model.predict(X_test)
print(f'{accuracy_score(y_test, y_predict_lr)}\n')

Naive Bayesmodel validating
0.768521875

Logistic Regression Model validating
0.78770625

